In [2]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv

# load secrets
load_dotenv()

True

In [3]:
# intialiaze api key
iex_key = os.getenv('IEX_API_KEY')

# IEX Cloud api query function to fetch historical data for stock symbol x
"""
think about scaling this function elswhere if needed.
when application expands -> can also break out into a .py file or a *class*  
"""

def get_historical_data(symbol, token):
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/chart/1y?token={iex_key}'
    response = requests.get(url)
    data = response.json() # get json response
    df = pd.DataFrame(data) # convert to dataframe
    df['date'] = pd.to_datetime(df['date']) # convert date to datetime
    df.set_index('date', inplace=True) # set date as index
    df.sort_index(inplace=True) # ensure data is sorted by date
    return df


In [4]:
# get historical data for Palantir (PLTR)
df_pltr = get_historical_data('PLTR', iex_key)

print(df_pltr.head())

            close   high    low  open   priceDate symbol    volume  \
date                                                                 
2022-11-07   7.02  7.850  6.890  7.78  2022-11-07   PLTR  99332839   
2022-11-08   7.21  7.420  6.910  7.08  2022-11-08   PLTR  56353934   
2022-11-09   6.98  7.185  6.950  7.12  2022-11-09   PLTR  42517300   
2022-11-10   7.64  7.970  7.430  7.46  2022-11-10   PLTR  58579775   
2022-11-11   8.41  8.420  7.375  7.48  2022-11-11   PLTR  55324135   

                           id   key subkey  ...   uLow   uVolume  fOpen  \
date                                        ...                           
2022-11-07  HISTORICAL_PRICES  PLTR         ...  6.890  99332839   7.78   
2022-11-08  HISTORICAL_PRICES  PLTR         ...  6.910  56353934   7.08   
2022-11-09  HISTORICAL_PRICES  PLTR         ...  6.950  42517300   7.12   
2022-11-10  HISTORICAL_PRICES  PLTR         ...  7.430  58579775   7.46   
2022-11-11  HISTORICAL_PRICES  PLTR         ...  7.375  553

In [5]:
# look at the data 
"""
the iex cloud api must have a default limit, but 2022 is perfect i don't want more than that
"""

# df_pltr
df_pltr.head()

,close,high,low,open,priceDate,symbol,volume,id,key,subkey,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
date,,,,,,,,,,,,,,,,,,,,,
2022-11-07,7.02,7.850,6.890,7.78,2022-11-07,PLTR,99332839,HISTORICAL_PRICES,PLTR,,...,6.890,99332839,7.78,7.02,7.850,6.890,99332839,"Nov 7, 22",0.00,0.0000
2022-11-08,7.21,7.420,6.910,7.08,2022-11-08,PLTR,56353934,HISTORICAL_PRICES,PLTR,,...,6.910,56353934,7.08,7.21,7.420,6.910,56353934,"Nov 8, 22",0.19,0.0271
2022-11-09,6.98,7.185,6.950,7.12,2022-11-09,PLTR,42517300,HISTORICAL_PRICES,PLTR,,...,6.950,42517300,7.12,6.98,7.185,6.950,42517300,"Nov 9, 22",-0.23,-0.0319
2022-11-10,7.64,7.970,7.430,7.46,2022-11-10,PLTR,58579775,HISTORICAL_PRICES,PLTR,,...,7.430,58579775,7.46,7.64,7.970,7.430,58579775,"Nov 10, 22",0.66,0.0946
2022-11-11,8.41,8.420,7.375,7.48,2022-11-11,PLTR,55324135,HISTORICAL_PRICES,PLTR,,...,7.375,55324135,7.48,8.41,8.420,7.375,55324135,"Nov 11, 22",0.77,0.1008
